In [ ]:
conda install -c anaconda pandas

# Create an awk function to select the gene id and first transcript of each gene
#figure out which column numbers are geneid and transcript in
#select the transcripts of the file only
awk '$3== "transcript"' stringtie_merged.gtf > StrTranscript_list.txt

# work on the first few lines
nawk -F, '{if(NR<=10) {print$1}}' StrTranscript_list.txt

#select the gene id's and the transcript id's from the txt using awk column select | get rid of the " ; so that only the id's are left
awk '{print $10"\t"$12}' StrTranscript_list.txt | sed 's/"//g;s/;//g'^C
sed -i 's/\t/,/g' replaced.txt


In [ ]:
import pandas as pd
# download txt as a dataframe 
table = pd.read_csv('/RNA/VALENCIA/replaced.txt', sep=",", header=None)
table

In [ ]:
# rename the column names of the dataframes and save as csv
table.columns = ['geneID','trID']

In [ ]:
# sort the gene id's and select the unique values
table = table.drop_duplicates(subset='geneID')
table

In [ ]:
#3) output all the genes' frst transcript sequences using blastdbcmd
# ~in jupyter~ output the 2nd column of transcript ID's as a list/txt file
table['transcriptID'].to_csv('/RNA/VALENCIA/table.txt', index = False, header = False )

# create the blastdatabse by referencing the list/txt file created from python
blastdbcmd -entry_batch table.txt -db /RNA/VALENCIA/mask_transcripts.fa -outfmt "%f" -out csi_blast.fa

In [ ]:
#2)create a blastdb using the newly created fastafile
#creating a masked database to protect the original fasta file
dustmasker -in ccl.transcripts.fa -infmt fasta -parse_seqids -outfmt maskinfo_asn1_bin -out ccl.transcript_dust.asnb

#create a database from the maseked file
makeblastdb -in ccl.transcripts.fa -input_type fasta -dbtype nucl -parse_seqids -mask_data ccl.transcript_dust.asnb -out mask_ccl.transcripts.fa -title "Citrus Clementine Chr"

In [ ]:
#3) blastn the csi sequence to the clementine database genes
blastn -db /RNA/VALENCIA/mask_ccl.transcripts.fa -query csi_blast.fa -outfmt 7 -out results.out

# filter the blast with >90% identity, alignment_length>100bp, 
blastn -db /RNA/VALENCIA/mask_ccl.transcripts.fa -query csi_blast.fa -num_threads 12 -outfmt 6 -out results.out

# column names
query acc.ver, subject acc.ver, % identity, alignment length, mismatches, gap opens, q. start, q. end, s. start, s. end, evalue, bit score

In [ ]:
#4) export the results to pandas df so that it can filtered with 90% identity, alignment_length>100bp and the best bitscore
results = pd.read_csv('/RNA/VALENCIA/results.out', sep = '\t', header = None)
results.columns = ['query','subject','% identity','length','mismatches','gaps','q.start','q.end','s.start','s.end','eval','bit']

In [ ]:
# filter the results
filter_res = results.loc[(results['% identity']>=90) & (results['length']>=100)] 
# drop duplicates
filter_res = filter_res.drop_duplicates(subset='query')
filter_res


In [ ]:
#export as csv
filter_res.to_csv('/RNA/VALENCIA/filter_res.csv', index = False)

In [ ]:
import pandas as pd
# match the id's found in upregGHA_PFA with the id's created frome filter_results
# import the files as dataframes
df = pd.read_csv('/RNA/VALENCIA/test.txt', sep ='\t', header = None)
df.columns = ['trID','subjID']
ghalist = pd.read_csv('/RNA/quants/upregGHA_PFA.csv')
ghalist.columns = ['geneID', 'log2FC', 'pval']

In [ ]:
merge = df.merge(table, on='trID', how='inner')
merge

In [ ]:
list = merge.merge(ghalist, on='geneID', how='inner')
list

In [ ]:
# change the subjID to match the required ID for AgriGO
list['subjID'] = list['subjID'].str.replace('\.v1.0','.g')
list['subjID'].to_csv('/RNA/VALENCIA/list.txt', index = False, header = False)
list

In [ ]:
# do the same for the downregGHA_PFA genes
df = pd.read_csv('/RNA/VALENCIA/test.txt', sep ='\t', header = None)
df.columns = ['trID','subjID']
ghalist = pd.read_csv('/RNA/quants/downregGHA_PFA.csv')
ghalist.columns = ['geneID', 'log2FC', 'pval']

In [ ]:
merge = df.merge(table, on='trID', how='inner')
list = merge.merge(ghalist, on='geneID', how='inner')


In [ ]:
# change the subjID to match the required ID for AgriGO and output
list['subjID'] = list['subjID'].str.replace('\.v1.0','.g')
list['subjID'].to_csv('/RNA/VALENCIA/list.txt', index = False, header = False)

In [65]:
#repeat for all other compressions
df = pd.read_csv('/RNA/VALENCIA/test.txt', sep ='\t', header = None)
df.columns = ['trID','subjID']
ghalist = pd.read_csv('/RNA/quants/upreg108_PV5.csv')
ghalist.columns = ['geneID', 'log2FC', 'pval']

ghalistdown = pd.read_csv('/RNA/quants/downreg108_PV5.csv')
ghalistdown.columns = ['geneID', 'log2FC', 'pval']

list = merge.merge(ghalist, on='geneID', how='inner')
listdown = merge.merge(ghalistdown, on='geneID', how='inner')

# change the subjID to match the required ID for AgriGO and output
list['subjID'] = list['subjID'].str.replace('\.v1.0','.g')
list['subjID'].to_csv('/RNA/VALENCIA/list.txt', index = False, header = False)

listdown['subjID'] = listdown['subjID'].str.replace('\.v1.0','.g')
listdown['subjID'].to_csv('/RNA/VALENCIA/listdown.txt', index = False, header = False)